In [52]:

import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import numpy as np
def warn(*args, **kwargs):
    pass
import warnings
warnings.warn = warn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
# imprt tree
from sklearn import tree
# import metrics
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder

In [53]:
df_raw = pd.read_csv("./labeled_data/labeled_df_15.csv", encoding_errors="ignore", on_bad_lines='skip', sep=",", index_col=False,
                    usecols=['SessionID','Activity','anomaly'])
# df_raw["TIMESTAMP"] = pd.to_datetime(df_raw["TIMESTAMP"], infer_datetime_format=True)
# df_raw = df_raw.sort_values(by=["SessionID", "TIMESTAMP"]).copy()
df_raw

,SessionID,Activity,anomaly
0,46,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,False
1,46,/werk_nl/werknemer/mijn_werkmap/doorgeven/mijn...,False
2,46,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_d...,False
3,46,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_b...,False
4,46,/werk_nl/werknemer/mijn_werkmap/postvak/mijn_b...,False
...,...,...,...
7174929,55314751,/werk_nl/werknemer/werkmap,False
7174930,55314751,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,False
7174931,55314751,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,False
7174932,55314751,/werk_nl/werknemer/mijn_werkmap/doorgeven/taken,False


In [54]:
# change anomaly == True to 1 and False to 0
df_raw["anomaly"] = df_raw["anomaly"].astype(int)

In [55]:
# count the number of unique SessionID where anomaly == True
count_anomaly_raw = df_raw[df_raw["anomaly"] == 1]["SessionID"].nunique()
count_normal_raw = df_raw[df_raw["anomaly"] == 0]["SessionID"].nunique()
print(f'Amount of anomalous sessions in the dataset:   {count_anomaly_raw}' )
print(f'Amount of normal sessions in the dataset:      {count_normal_raw}')
distribution =  count_anomaly_raw / count_normal_raw 
print(f'Distribution:                                  {distribution * 100} %')

Amount of anomalous sessions in the dataset:   355
Amount of normal sessions in the dataset:      659915
Distribution:                                  0.05379480690695014 %


In [56]:
df_anomaly = df_raw[df_raw["anomaly"] == 1].copy()
df_normal = df_raw[df_raw["anomaly"] == 0].copy()
nr_of_sessions_used = 50000
injection_rate = nr_of_sessions_used / count_normal_raw
injection_amount = int(injection_rate * count_anomaly_raw)
print(injection_rate)
print(injection_amount)
# get 20 random sessionIDs from the anomaly dataset
anomaly_sessionIDs = df_anomaly["SessionID"].sample(n=injection_amount, random_state=1).tolist()




0.07576733367176075
26


In [57]:
# pick random int > 0 and < (len(df_raw) - nr_of_sessions_used)
lower = np.random.randint(0, len(df_raw) - nr_of_sessions_used)
upper = lower + nr_of_sessions_used
print(f'Lower bound: {lower}')
print(f'Upper bound: {lower + nr_of_sessions_used}')
# get a random sample of the sessions
# df_ = df_raw[df_raw["SessionID"].isin(df_raw["SessionID"].unique()[:nr_of_sessions_used])].copy()
df_ = df_normal[df_normal["SessionID"].isin(df_normal["SessionID"].unique()[:nr_of_sessions_used])].copy()
df_ = df_.append(df_anomaly[df_anomaly["SessionID"].isin(anomaly_sessionIDs)], ignore_index=True)
# Only use the columns that are needed -> SessionID, Activity, anomaly
df_50k = df_[["SessionID", "Activity", "anomaly"]]


# remove the sessions with ID in anomaly_sessionIDs from the df_anomaly dataset
df_anomaly = df_anomaly[~df_anomaly["SessionID"].isin(anomaly_sessionIDs)].copy()
df_anomaly.to_csv("gen_sessions/an.csv", index=False)

Lower bound: 1550847
Upper bound: 1600847


In [58]:
# count the number of unique SessionID where anomaly == True
count_anomaly = df_50k[df_50k["anomaly"] == 1]["SessionID"].nunique()
count_normal = df_50k[df_50k["anomaly"] == 0]["SessionID"].nunique()
print(f'Amount of anomalous sessions in the sampled dataset:   {count_anomaly}')
print(f'Amount of normal sessions in the sampled dataset:      {count_normal}')
distribution =  count_anomaly / count_normal 
print(f'Distribution:                                          {distribution * 100} %')

Amount of anomalous sessions in the sampled dataset:   26
Amount of normal sessions in the sampled dataset:      50000
Distribution:                                          0.052 %


Helper Functions

In [59]:
# function to encode the activities
def encode_list(_list:list) -> list:
    _l = _list.copy()
    le = LabelEncoder()
    le.fit(_l)
    l= le.transform(_l)
    return l

# function to create a list of al the activities
def activities_list(_df):
  df = _df.copy()
  activities = []
  for i in df["Activity"].unique():
    activities.append(i)
  return activities

# function to create a list of all the transitions
def get_all_transitions(_df):
  df = _df.copy()
  transitions = []
  # create consecutive column
  df['consecutive'] = df.groupby("SessionID")['Activity'].shift(-1)
  # if consecutive is NaN, then consecutive = END
  df['consecutive'] = df['consecutive'].fillna("END")
    # create column with the transition
  df['transition'] = df['Activity'] + "->" + df['consecutive']
  # get all unique transitions en put in transitions list
  transitions = df['transition'].unique().tolist()
  return transitions
  

# function to create a dictionary with the activities as keys and the one hot encoded activities as values
def create_dict(l1, l2):
  dic = {}
  for i in range(len(l1)):
    dic[l1[i]] = l2[i]
  return dic

# function to transform each session into a sequence of activities
def transform_session_to_sequence(_df):
  df = _df.copy()
  df_ses = df.groupby("SessionID")["Activity"].apply(list).reset_index()
  # merge the anomaly column
  df_ses = df_ses.merge(df[["SessionID", "anomaly"]].drop_duplicates(), on="SessionID")
  return df_ses

# function the create the dataFrame for the model
def df_for_model(_df,Activity_col):
  df = pd.DataFrame(_df[Activity_col].values.tolist(), index= _df.index)
  df_for_model = pd.concat([_df, df], axis=1)
  df_for_model = df_for_model.drop(columns=[Activity_col])
  df_for_model.fillna(0, inplace=True)
  return df_for_model
  
# function to concat 2 dataframes
def concat_df(df_gen ,amount_real,amount_generated,_df1 = df_raw,):
  df1 = _df1.copy()
  df2 = df_gen
  df_raw_filtered = df1[df1["SessionID"].isin(df1["SessionID"].unique()[:amount_real])]
  df_gen_anomalies_filtered = df2[df2["SessionID"].isin(df2["SessionID"].unique()[:amount_generated])]
  df = pd.concat([df_raw_filtered, df_gen_anomalies_filtered], ignore_index=True)
  return df

GLOBALS

In [60]:
# Global variables
# list of all possible activities in df_raw
ALL_ACTIVITIES = activities_list(df_raw)
ENCODED_ACTIVITIES = encode_list(ALL_ACTIVITIES)

ALL_TRANSITIONS = get_all_transitions(df_raw)
ENCODED_TRANSITIONS = encode_list(ALL_TRANSITIONS)

DICT = create_dict(ALL_ACTIVITIES, ENCODED_ACTIVITIES)
DICT_TRANSITIONS = create_dict(ALL_TRANSITIONS, ENCODED_TRANSITIONS)

if len(ALL_ACTIVITIES) == len(ENCODED_ACTIVITIES) == len(DICT) == len(ALL_TRANSITIONS) == len(ENCODED_TRANSITIONS):
  print("All lists are the same length")


Stappenplan:

1. df -> transform_session_to_sequence
2. Transform Activities to encoded Activities
3. Transform the list in the Activity column to multiple columns with fillna(0)

In [61]:
base_data = transform_session_to_sequence(df_50k)
# base_an = transform_session_to_sequence(df_50k)
base_data

,SessionID,Activity,anomaly
0,46,[/werk_nl/werknemer/mijn_werkmap/doorgeven/tak...,0
1,92,[/werk_nl/werknemer/mijn_werkmap],0
2,358,[/werk_nl/werknemer/mijn_werkmap/postvak/mijn_...,0
3,420,[/werk_nl/werknemer/mijn_werkmap/doorgeven/tak...,0
4,447,[/werk_nl/werknemer/mijn_werkmap/postvak/mijn_...,0
...,...,...,...
50021,42771301,[/werk_nl/werknemer/mijn_werkmap/werk-zoeken/v...,1
50022,45419685,"[/werk_nl/werknemer/home, /werk_nl/werknemer/e...",1
50023,46475745,"[/werk_nl/werknemer/contact, /portal/page/port...",1
50024,50623714,"[/werk_nl/werknemer/home, /werk_nl/werknemer/m...",1


In [62]:
# function that combines a item in a list with the next item
def list_combine(_list):
  list_ = _list.copy()
  res= []
  for index,item in enumerate(list_):
    if index < len(list_)-1:
      res.append(item + "->" + list_[index+1])
    else:
      res.append(item + "->" + "END")
  return res


In [63]:
# base_data['Activity'] = base_data['Activity'].apply(list_combine)

In [64]:
base_data['Activity']

0        [/werk_nl/werknemer/mijn_werkmap/doorgeven/tak...
1                        [/werk_nl/werknemer/mijn_werkmap]
2        [/werk_nl/werknemer/mijn_werkmap/postvak/mijn_...
3        [/werk_nl/werknemer/mijn_werkmap/doorgeven/tak...
4        [/werk_nl/werknemer/mijn_werkmap/postvak/mijn_...
                               ...                        
50021    [/werk_nl/werknemer/mijn_werkmap/werk-zoeken/v...
50022    [/werk_nl/werknemer/home, /werk_nl/werknemer/e...
50023    [/werk_nl/werknemer/contact, /portal/page/port...
50024    [/werk_nl/werknemer/home, /werk_nl/werknemer/m...
50025    [/werk_nl/werknemer/home, /werk_nl/werknemer/m...
Name: Activity, Length: 50026, dtype: object

In [65]:
base_data['Activity'] = base_data['Activity'].apply(lambda x: [DICT[i] for i in x])
# base_an['Activity'] = base_an['Activity'].apply(lambda x: [DICT[i] for i in x])

In [66]:
base_data

,SessionID,Activity,anomaly
0,46,"[372, 371, 392, 391, 391, 391]",0
1,92,[369],0
2,358,"[391, 372, 401, 391, 391, 391, 391, 369, 367]",0
3,420,"[372, 372, 372, 372, 372, 372]",0
4,447,"[391, 391, 391, 391]",0
...,...,...,...
50021,42771301,"[402, 402, 402, 402, 402, 402, 165, 367, 622, ...",1
50022,45419685,"[367, 365, 398, 365, 365, 398, 398, 398, 398, ...",1
50023,46475745,"[353, 41, 41, 367, 41, 41, 41, 41, 41, 41, 43,...",1
50024,50623714,"[367, 391, 392, 392, 372, 372, 372, 371, 371, ...",1


In [67]:
base_data = df_for_model(base_data, "Activity")
# base_an = df_for_model(base_an, "Activity")

In [68]:
base_data
# drop the SessionID column
base_data = base_data.drop(columns=["SessionID"])
base_data

,anomaly,0,1,2,3,4,5,6,7,8,...,355,356,357,358,359,360,361,362,363,364
0,0,372,371.0,392.0,391.0,391.0,391.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,369,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,391,372.0,401.0,391.0,391.0,391.0,391.0,369.0,367.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,372,372.0,372.0,372.0,372.0,372.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,391,391.0,391.0,391.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50021,1,402,402.0,402.0,402.0,402.0,402.0,165.0,367.0,622.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50022,1,367,365.0,398.0,365.0,365.0,398.0,398.0,398.0,398.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50023,1,353,41.0,41.0,367.0,41.0,41.0,41.0,41.0,41.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
50024,1,367,391.0,392.0,392.0,372.0,372.0,372.0,371.0,371.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Functions to run the model

In [69]:
# Function to split the data into train and test data
def split_data(_df):
  df = _df.copy()
  X = df.drop(columns=["anomaly"])
  y = df["anomaly"]
  X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=42)
  return X_train, X_test, y_train, y_test

In [70]:
# Base test and train data
X_train, X_test, y_train, y_test = split_data(base_data)
print(X_train)

       0      1      2      3      4      5      6      7      8      9    \
35064  372    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
14261  372  402.0  402.0  402.0  402.0  402.0  402.0  402.0  402.0  402.0   
34001  372    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
47759  542  372.0  374.0  374.0  374.0  374.0  369.0  635.0   39.0  367.0   
5325   391  367.0  542.0  391.0  391.0  391.0  391.0  367.0    0.0    0.0   
...    ...    ...    ...    ...    ...    ...    ...    ...    ...    ...   
11284  542  371.0  372.0  372.0  372.0  372.0  391.0  372.0  367.0    0.0   
44732  367  391.0  371.0  372.0  391.0  391.0  391.0  391.0  391.0  391.0   
38158  367  371.0  372.0  372.0  375.0  402.0  402.0  402.0  402.0    0.0   
860    369  369.0  369.0  369.0  369.0  369.0  506.0  369.0  398.0  398.0   
15795  372  391.0  391.0  392.0  371.0    0.0    0.0    0.0    0.0    0.0   

       ...  355  356  357  358  359  360  361  362  363  364  
35064  ...  

Run the model

In [71]:

# import the generated sessions:
ses_amount = 1000
base_path = f"gen_sessions/{str(ses_amount)}/"

gen_sessions_paths = [
  base_path + f'5_{ses_amount}.csv',
  base_path + f'10_{ses_amount}.csv',
  base_path + f'25_{ses_amount}.csv',
  base_path + f'50_{ses_amount}.csv',
  base_path + f'75_{ses_amount}.csv',
  base_path + f'100_{ses_amount}.csv',
  # base_path + 'an.csv',
  
  # base_path + '75_10000.csv',
  # base_path + '100_10000.csv',
]

In [72]:
auc_score_dicts = {}
dict_prec_and_rec_list = {}

amount_anomalies_list_total = []
precision_score_list_total = []
recall_score_list_total = []

In [75]:
from sklearn.metrics import accuracy_score, balanced_accuracy_score, precision_score, recall_score, roc_auc_score


for sessions in gen_sessions_paths:
    amount_anomalies_list = []
    precision_score_list = []
    recall_score_list = []
    
    # build the dataFrame
    cvs = pd.read_csv(sessions,index_col=False)
    # check if column Unnamed: 0 exists
    if "Unnamed: 0" in cvs.columns : cvs = cvs.drop(columns=["Unnamed: 0"])
    cvs['anomaly'] = 1
    cvs.rename(columns={'URL_FILE':'Activity'}, inplace=True)
    gen_df_seq = transform_session_to_sequence(cvs)

     
    # gen_df_seq['Activity'] = gen_df_seq['Activity'].apply(list_combine)
        
    gen_df_seq['Activity'] = gen_df_seq['Activity'].apply(lambda x: [DICT[i] for i in x])
    
    ready_df = df_for_model(gen_df_seq, "Activity")
    # ready_df = gen_df_seq
    ready_df = ready_df.drop(columns=["SessionID"])
    ready_df = ready_df.sample(frac=1).reset_index(drop=True)
     
    
    for amount_gen in [10, 20, 50, 65, 100, 200, 300, 400, 500, 600, 700, 800, 900, 1000]:
    # 0 to 10000 in 250
    # for amount_gen in range(0, 5000, 250):
    # from 0 to 250 with a step of 25
    # for amount_gen in [0, 25, 50, 75, 100, 125, 150, 175, 200, 225, 250]:
        amount_real = 0
        df_gen = ready_df[:amount_gen]
         
        # Add the generated anomalies to the training dataset
        X_train_extra = pd.concat([X_train, df_gen.drop(columns=["anomaly"])]).fillna(0)
        y_train_extra = pd.concat([y_train, df_gen["anomaly"]])
        
        clf = tree.DecisionTreeClassifier().fit(X_train_extra, y_train_extra)
        # clf_extra = LogisticRegression(random_state=0, max_iter=1000).fit(X_train_extra, y_train_extra_transformed)
        #y_train_predict = clf_extra.predict(X_train_extra)
        predictions = clf.predict(X_test)
        
        print(sessions)
        print(amount_gen)
        print(accuracy_score(y_test, predictions))
        print(balanced_accuracy_score(y_test, predictions))
        print()
    
       
        


gen_sessions/5000/5_5000.csv
10
0.9985008994603238
0.5994500549945005

gen_sessions/5000/5_5000.csv
20
0.9985008994603238
0.4995000499950005

gen_sessions/5000/5_5000.csv
50
0.997101738956626
0.5987501249875012

gen_sessions/5000/5_5000.csv
65
0.9972016789926045
0.4988501149885011

gen_sessions/5000/5_5000.csv
100
0.9960023985608635
0.49825017498250174

gen_sessions/5000/5_5000.csv
200
0.9934039576254248
0.49695030496950304

gen_sessions/5000/5_5000.csv
300
0.9911053367979212
0.5957504249575043

gen_sessions/5000/5_5000.csv
400
0.9900059964021587
0.49525047495250474

gen_sessions/5000/5_5000.csv
500
0.9866080351788926
0.5935006499350065

gen_sessions/5000/5_5000.csv
600
0.9878073156106336
0.5941005899410059

gen_sessions/5000/5_5000.csv
700
0.9868079152508495
0.49365063493650635

gen_sessions/5000/5_5000.csv
800
0.9844093543873675
0.6923507649235077

gen_sessions/5000/5_5000.csv
900
0.9825104937037777
0.6914008599140087

gen_sessions/5000/5_5000.csv
1000
0.9810113931641016
0.7906009399